In [29]:
# typing — Support for gradual typing as defined by PEP 484 and subsequent PEPs.
from typing import TypeAlias, Literal, List, Tuple, Callable, Optional

# NumPy — Funções para computação numérica.
import numpy as np

# Numba — Aceleração do Python via JIT.
from numba import njit

# matplotlib — An object-oriented plotting library.
import matplotlib.pyplot as plt

# copy — Deep and shallow copies.
from copy import deepcopy as copy

In [30]:
@njit
def phi(x: np.ndarray, domain: Tuple[float, float]) -> float:
	
	# `nd` é o comprimento de `x`.
	nd = len(x)

	# `xmin` e `xmax` são os limites do domínio.
	xmin, xmax = domain[0], domain[1]
	
	# Esta é a fórmula de Φ(bj).
	return xmin + ((xmax - xmin) / (2 ** nd - 1)) * np.sum(x[::-1] * 2 ** np.arange(nd))

phi(np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]), (-10, 10))

0.0

In [31]:
@njit
def chunks(x: np.ndarray, p: int) -> np.ndarray:
	"""
	Retorna `x` em `p` "pedaços".
	"""
	
	m = x.size // p
	
	result = np.empty((p, m), dtype = x.dtype)
	
	for i in range(p):
		result[i] = x[i * m: (i + 1) * m]

	return result

chunks(np.array([0, 1, 1, 1, 1, 0]), 2)

array([[0, 1, 1],
       [1, 1, 0]])

In [32]:
@njit
def phi_nd(x: np.ndarray, p: int, domain: Tuple[float, float]) -> np.ndarray:
    """
    Aplica `phi(...)` em cada `nd` do indivíduo.
    """
    return np.array([phi(chunk, domain) for chunk in chunks(x, p)])

phi_nd(np.array([1, 1, 0, 0, 1, 0]), 3, (-10, 10))

array([ 10.        , -10.        ,   3.33333333])

In [33]:
@njit
def rastrigin(x: np.ndarray) -> np.signedinteger:
	
	# `n` é o comprimento do vetor `x`.
	n = len(x)

	# Constante: A
	A = 10

	# Esta é a função de Rastrigin, propriamente dita.
	return A * n + np.sum(x ** 2 - A * np.cos(2 * np.pi * x))

rastrigin(np.array([0, 0, 0, 0, 0, 0]))

0.0

In [101]:
def create_individual(size: int) -> np.ndarray:
	"""
	Cria um indivíduo aleatório.
	"""

	return np.random.randint(low = 0, high = 2, size = size)

create_individual(20)

array([0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1])

In [102]:
@njit
def roulette_selection(population: np.ndarray, fitnesses: np.ndarray) -> np.ndarray:
	
	# Valores menores são preferidos
	inverted_fitnesses = 1.0 / (fitnesses + 1e-10) # Evitar divisões por zero.

	# Este é o total de todas as aptidões.
	total_fitness = np.sum(inverted_fitnesses)
	
	# Crie uma distribuição de probabilidades.
	probabilities = inverted_fitnesses / total_fitness
	
	# Select an individual based on the probability distribution.
	cumulative_probabilities = np.cumsum(probabilities)
	
	# Gere um número aleatório entre [0, 1).
	r = np.random.rand()
	
	# Select the individual.
	for i, individual in enumerate(population):
		
		# Caso o `r` for menor que a probabilidade cumulativa..
		if r < cumulative_probabilities[i]:
			
			# ..retorne o indivíduo.
			return individual

In [103]:
@njit
def roulette_selection(population: np.ndarray, fitnesses: np.ndarray) -> np.ndarray:
	
    # Calcule as probabilidades
	probabilities = 1 / (fitnesses / np.sum(fitnesses))
	
	# Inicalize o índice.
	i = 0
	
	# Compute o total das probabilidades a partir do primeiro.
	total = probabilities[i]
	
	# Produza um número aleatório entre [0, 1).
	r = np.random.rand()
	
	# Rode a roleta!
	while total < r:
		i += 1
		total += probabilities[i]
	
	# Retorne o indivíduo.
	return population[i]

In [104]:
@njit
def crossover(a: np.ndarray, b: np.ndarray, rate: float) -> Tuple[np.ndarray, np.ndarray]:
	
	# Faça uma cópia dos dois.
	os_a = a.copy()
	os_b = b.copy()
	
	if np.random.rand() < rate:
		
		# Crie uma máscara "aleatória".
		mask = np.random.randint(0, 2, a.size)

		# De bit em bit..
		for i in range(a.size):
			
			# Troque os genes dos indivíduos.
			if mask[i] == 1:
				os_a[i], os_b[i] = b[i], a[i]
				
	return os_a, os_b

In [105]:
@njit
def mutate(individual: np.ndarray, mutation_rate: float) -> np.ndarray:
	"""
	Realiza a mutação em um indivíduo.
	"""

	# Para cada bit do indivíduo..
	for i in range(individual.size):
		
		# "Tógle" o bit.
		if np.random.rand() < mutation_rate:
			individual[i] = not individual[i]

	return individual

In [106]:
@njit
def has_converged(population: np.ndarray) -> bool:
	"""
	A população convergiu caso todos sejam o mesmo indivíduo.
	"""

	first = population[0]
	
	for i in range(1, population.shape[0]):
		if not np.array_equal(first, population[i]):
			return False
		
	return True

has_converged(np.array([
	np.array([0, 1, 1, 0]),
	np.array([0, 1, 1, 0])
]))

True

In [107]:
def genetic_algorithm(P: int, p: int, nd: int, domain: Tuple[float, float], generations: int = 1_000, rounds: int = 100, crossover_rate: float = 0.9, mutation_rate: float = 0.01) -> List[float]:
	
	# Lista de todas as aptidões.
	all_fitnesses = []

	for _ in range(rounds):

		# População inicial aleatória.
		population = np.array([create_individual(p * nd) for _ in range(P)])

		for _ in range(generations):

			# Esta é uma lista de todas as aptidões.
			fitnesses = np.array([rastrigin(phi_nd(individual, p, domain)) for individual in population])

			# Ordene a população pela aptidão.
			sorted_indices = np.argsort(fitnesses)
			population = population[sorted_indices]
			fitnesses = fitnesses[sorted_indices]

			# ELITISMO: Coloque os 10 melhores da população anterior na nova população.
			new_population = population[0 : 25].tolist()
			
			while len(new_population) < P:
					
				# Selecione dois indivíduos com o método da roleta.
				parent1 = roulette_selection(population, fitnesses)
				parent2 = roulette_selection(population, fitnesses)
				
				# Estes são sua próle.
				offspring1, offspring2 = crossover(parent1, parent2, crossover_rate)
				
				# Faça uma mutação nos indivíduos.
				offspring1 = mutate(offspring1, mutation_rate)
				offspring2 = mutate(offspring2, mutation_rate)

				# Adicione as próles à nova população.
				new_population.append(offspring1)
				new_population.append(offspring2)
			
			# A nova população substitui a "antiga".
			population = np.array(new_population)

			# Adicione a melhor aptidão às melhores aptidões.
			all_fitnesses += fitnesses.tolist()

		# Caso a população tenha convergido, pare.
		if has_converged(population):
			break

	return all_fitnesses

In [134]:
fitnesses = genetic_algorithm(
	
	# Definições do Problema.
	p = 20,
	domain = (-10, 10),
    
	# Rodadas totais.
	rounds = 100,
	
	# Hiperparâmetros Gerais.
	generations = 100,
	P = 50,
	nd = 8,

	# Hiperparâmetros Específicos.
	crossover_rate = 0.85,
	mutation_rate = 0.01
	
)

In [135]:
print("Melhor Aptidão:", np.min(fitnesses))
print("Pior Aptidão:", np.max(fitnesses))
print("Aptidão Média:", np.mean(fitnesses))
print("Desvio Padrão das Aptidões:", np.std(fitnesses))

Melhor Aptidão: 33.69395339227256
Pior Aptidão: 1379.8769635522067
Aptidão Média: 168.9071210994011
Desvio Padrão das Aptidões: 132.84725127010722
